# Basic Usage

## Imports

In [1]:
from autora.theorist.rnn_sindy_rl import RNNSindy
from autora.state import StandardState, on_state, Delta
from autora.variable import VariableCollection, Variable

import numpy as np
import pandas as pd

## Constants

In [2]:
TRIALS_PER_PARTICIPANT = 30

## Set up variables
independent variable is "reward-trajectory": A 2 x n_trials Vector with entries between 0 and 1
dependent variable is "choice-trajectory": A 2 x n_trials Vector with boolean entries (one hot encoded)

In [3]:
variables = VariableCollection(
    independent_variables=[Variable(name="reward-trajectory")],
    dependent_variables=[Variable(name="choice-trajectory")]
)

# State
We use the StandardState

In [4]:
state = StandardState(variables=variables)

## Components
### Experimentalist
Our experimentalist creates n random reward-trajectories

In [5]:
@on_state()
def experimentalist_random_trajectory_on_state(num_samples, n_trials=TRIALS_PER_PARTICIPANT):
    """
    This is creates `num_samples` randomized reward trajectories of length `n_trials`
    """
    trajectories = np.random.randint(0, 2, (num_samples, n_trials, 2))
    trajectories_list = [trajectory for trajectory in trajectories]
    conditions = pd.DataFrame({'reward-trajectory': trajectories_list})

    return Delta(conditions=conditions)

For example, let's create 3 conditions:

In [6]:
state = experimentalist_random_trajectory_on_state(state, num_samples=3)
state.conditions

,reward-trajectory
0,"[[0, 1], [1, 1], [0, 0], [0, 0], [0, 1], [0, 0..."
1,"[[1, 0], [0, 1], [0, 1], [1, 0], [1, 0], [0, 1..."
2,"[[0, 1], [1, 0], [0, 0], [1, 1], [1, 1], [1, 1..."


## Experiment Runner
Here, we create a rando choice trajectory

In [7]:
@on_state()
def runner_on_state(conditions):
    def generate_choice():
        return [0, 1] if np.random.rand() > 0.5 else [1, 0]

    trajectories_list = [
        np.array([generate_choice() for _ in range(len(reward_trajectory))])
        for reward_trajectory in conditions['reward-trajectory']
    ]
    experiment_data = pd.DataFrame({
        'reward-trajectory': conditions['reward-trajectory'].tolist(),
        'choice-trajectory': trajectories_list}, dtype=object)
    return Delta(experiment_data=experiment_data)

In [8]:
state = runner_on_state(state)
state.experiment_data

,reward-trajectory,choice-trajectory
0,"[[0, 1], [1, 1], [0, 0], [0, 0], [0, 1], [0, 0...","[[1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1..."
1,"[[1, 0], [0, 1], [0, 1], [1, 0], [1, 0], [0, 1...","[[0, 1], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0..."
2,"[[0, 1], [1, 0], [0, 0], [1, 1], [1, 1], [1, 1...","[[1, 0], [0, 1], [0, 1], [0, 1], [0, 1], [1, 0..."


## Theorist

We use the RNNSynd regressor to fit the data:

In [9]:
regressor = RNNSindy(2)
@on_state()
def theorist_on_state(X, y):
    return Delta(models=[regressor.fit(X, y)])

In [10]:
state = theorist_on_state(state)
state.models[-1]    

Training the RNN...
Epoch 1/100 --- Loss: 14.2566595; Time: 0.3946s; Convergence value: 1.33e+01
Epoch 2/100 --- Loss: 5.2606778; Time: 0.3832s; Convergence value: 1.11e+01
Epoch 3/100 --- Loss: 5.0337067; Time: 0.4014s; Convergence value: 7.37e+00
Epoch 4/100 --- Loss: 6.1062818; Time: 0.1255s; Convergence value: 5.72e+00
Epoch 5/100 --- Loss: 2.6393573; Time: 0.5418s; Convergence value: 5.24e+00
Epoch 6/100 --- Loss: 5.2130003; Time: 0.1352s; Convergence value: 4.76e+00
Epoch 7/100 --- Loss: 2.9188631; Time: 0.1499s; Convergence value: 4.37e+00
Epoch 8/100 --- Loss: 2.3886142; Time: 0.1742s; Convergence value: 3.83e+00
Epoch 9/100 --- Loss: 0.1580222; Time: 0.1679s; Convergence value: 3.63e+00
Epoch 10/100 --- Loss: 0.9369656; Time: 0.1758s; Convergence value: 3.30e+00
Epoch 11/100 --- Loss: 1.3392670; Time: 0.1337s; Convergence value: 2.98e+00
Epoch 12/100 --- Loss: 7.0660563; Time: 0.1777s; Convergence value: 3.24e+00
Epoch 13/100 --- Loss: 0.4229920; Time: 0.2017s; Convergence val

RNNSindy()